Kenneth Cole Davis 
This is the main file for my project which shows each step of the project.
The supporting files are simpleKNN.py, KNN.py, simplePerceptron.py, analytics.py, and iris.data

simpleKNN.py:
    Is used in evaluate my simple data set which has two features and only two choices of labels (0 or 1).

analytics.py
    Is used to find the Accuracy, Precision, Recall, F1-score (macro/weighted) of the simple KNN algorithm.

KNN.py:
    Is used in evaluating the IRIS data set which has four featuers and three posible labels. There will be 150 entries split into a random 60/20/20 testing,validation, testing split. The validation data will be used to chose an optimal K and then used on testing data. We will also test two distance metrics, euclidean and manhattan. Finally, the functions used to calculate Accuracy, Precision, Recall, F1-score (macro/weighted) are also in this class.

simplePerceptron.py:
    Is used to do a simple version of the peceptron on the same simple data set from simple KNN



In [1]:
## Headers
from simpleKNN import simpleKNN
import pandas as pd
from analytics import analytics
from KNN import KNN
from simplePerceptron import perceptron


In [2]:
## Declare sample data 
simple_training_data = [
    ([2,1], 1), ([2,2], 1), ([3,2], 1), ([3,3], 1),      ## Label 1
    ([-1,-1], 0), ([-2,-1], 0), ([-2,-2], 0), ([0,-3],0)   # Label 0
]

simple_testing_data = [
    ([2,2],1), ([2,3],0), ([-1,-3],0), ([-3,-3], 1), ([4,2], 1)
]

In [3]:
## fill in each data points prediction
k = 3
allpredictions = []
for testPoint in simple_testing_data:
    allpredictions.append(simpleKNN.predict_classification(simple_training_data, testPoint, k))
print("The predictions for the test data are", allpredictions)


The predictions for the test data are [1, 1, 0, 0, 1]


Here is the implementation of KNN from scratch on a simple tiny dataset:

In [4]:
rows = []
for test_point in simple_testing_data:
    x, y_true = test_point
    y_pred = simpleKNN.predict_classification(simple_training_data,test_point,k)
    rows.append({
        "test_point": tuple(x),
        "label":int(y_true),
        "prediction": int(y_pred)
})

results = pd.DataFrame(rows, columns = ["test_point", "label", "prediction"])
display(results)

,test_point,label,prediction
0,"(2, 2)",1,1
1,"(2, 3)",0,1
2,"(-1, -3)",0,0
3,"(-3, -3)",1,0
4,"(4, 2)",1,1


Here is the confusion matrix and the corresponding accuracy, precision, recall, F1-score (macro/weighted).

Accuracy is only so low here becuase of the test data I chose, where there is 2 points that cleary fit with the majority of points around them, they just happen to have different labels. I did this primarily to make sure the confusion matrix was calculating correctly.

In [5]:
##Check confusion matrix
a = analytics(results)
confusion_matrix = a.confusion_matrix()
print("Total points:", a.total_points)
print("Confusion Matrix is: \n", confusion_matrix)
print("\nTP:", a.tp, "TN:", a.tn, "FN:", a.fn, "FP:", a.fp)

## Calculate Test values for 
print("precision =", a.get_precision())
print("accuracy = ", a.get_accuracy())
print("recall =", a.get_recall())
print("F1 Score =", a.get_f1_score())
print("F1 Score (Macro)", a.get_f1_macro())
print("F1 Score (Weighted)", a.get_f1_weighted())


Total points: 5
Confusion Matrix is: 
                         Actually Positive (1)  Actually Negative (0)
Predicted Positive (1)                      2                      1
Predicted Negative (0)                      1                      1

TP: 2 TN: 1 FN: 1 FP: 1
precision = 0.6666666666666666
accuracy =  0.6
recall = 0.6666666666666666
F1 Score = 0.6666666666666666
F1 Score (Macro) 0.5833333333333333
F1 Score (Weighted) 0.6


Here we download the IRIS dataset and take a peak at it

In [6]:
##Download the IRIS dataset from UCI Website (https://archive.ics.uci.edu/ml/datasets/iris)
## I downloded it from the website and will just import the CSV File

file_name = "iris.data"
columns = ["sepal_length","sepal_width","petal_length","petal_width","species"]
iris_df = pd.read_csv(file_name, header = None, names = columns)
print(iris_df)


     sepal_length  sepal_width  petal_length  petal_width         species
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
..            ...          ...           ...          ...             ...
145           6.7          3.0           5.2          2.3  Iris-virginica
146           6.3          2.5           5.0          1.9  Iris-virginica
147           6.5          3.0           5.2          2.0  Iris-virginica
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica

[150 rows x 5 columns]


Simple paragraph about the dataset:

The IRIS dataset above, is one of the earliest datasets used in classifications methods. It originates from UC Irvines' Machine Learning Repository. Each record in the dataset is representative of one instance of an iris plant, with its corresponding measurments and iris class. There is a total of 150 instances with four features for each, sepal_length, sepal_width, petal_length, and petal_width. For the three classes, Iris Setosa, Iris Versicolour, and Iris Virginica each has 50 instances recorded. One of the classes is said to be linearly seperable from the other two, the other two are not linearly seperable. All of the features are said to be continuous.

** All information above is from the UCI ML repository which is linked in the coding cell above this one **

This is the run through of knn on the iris data set with euclidean distance which will be comapred with manhattan distance afterwards

In [7]:
## Initalize chose variables and oject for advanced KNN on IRIS dataset  -- Euclidean distance 
from KNN import KNN
train_ratio=0.60
val_ratio=0.20
test_ratio = 0.20
k = 3
distance = 'euclidean'
knn_euclidean = KNN(iris_df, k, train_ratio, val_ratio, test_ratio, distance)      ##when this object is created, data is automatically split

In [8]:
# Validation stage - chose possible K values to validate and find best K
k_grid=(1,3,5,7,9)
best_k, val_acc = knn_euclidean.choose_best_k(k_grid)
print("Best K:", best_k, "\nValidation data accuracy with best K:", round(val_acc, 4))

Best K: 1 
Validation data accuracy with best K: 1.0


In [9]:
#Using the best K, get the results 
results, metrics = knn_euclidean.evaluate(best_k)
print("Using the best k, which is", best_k,", the results on the test data are as follows:")
print("For labels and predictions:")
print("0 = Iris-setosa\n1 = Iris-versicolor\n2 = Iris-virginica")
display(results)
print("Accuracy:", round(metrics["accuracy"], 4))
print("Precision:", round(metrics["precision"], 4))
print("Recall:", round(metrics["recall"], 4))
print("F1 (macro):", round(metrics["f1_macro"], 4), "&  F1 (weighted):", round(metrics["f1_weighted"], 4))

Using the best k, which is 1 , the results on the test data are as follows:
For labels and predictions:
0 = Iris-setosa
1 = Iris-versicolor
2 = Iris-virginica


,test point,label,prediction
0,"(5.9, 3.0, 5.1, 1.8)",2,2
1,"(6.9, 3.1, 4.9, 1.5)",1,1
2,"(5.3, 3.7, 1.5, 0.2)",0,0
3,"(6.3, 2.8, 5.1, 1.5)",2,1
4,"(6.1, 2.8, 4.7, 1.2)",1,1
5,"(6.8, 3.0, 5.5, 2.1)",2,2
6,"(5.0, 2.0, 3.5, 1.0)",1,1
7,"(6.4, 2.9, 4.3, 1.3)",1,1
8,"(5.2, 4.1, 1.5, 0.1)",0,0
9,"(7.9, 3.8, 6.4, 2.0)",2,2


Accuracy: 0.9667
Precision: 0.9667
Recall: 0.9667
F1 (macro): 0.9691 &  F1 (weighted): 0.9665


In [10]:
## Initalize chose variables and oject for advanced KNN on IRIS dataset  -- Manhattan distance 
from KNN import KNN
train_ratio=0.60
val_ratio=0.20
test_ratio = 0.20
k = 3
distance = 'manhattan'
knn_manhattan = KNN(iris_df, k, train_ratio, val_ratio, test_ratio, distance)

In [11]:
# Training stage - chose possible K values to validate and find best K
k_grid=(1,3,5,7,9)
best_k, val_acc = knn_manhattan.choose_best_k(k_grid)
print("Best K:", best_k, "\nValidation data accuracy with best K:", round(val_acc, 4))

Best K: 3 
Validation data accuracy with best K: 1.0


In [12]:
#Using the best K, get the results 
results, metrics = knn_manhattan.evaluate(best_k)
print("Using the best k, which is", best_k,", the results are as follows:")
print("For labels and predictions:")
print("0 = Iris-setosa\n1 = Iris-versicolor\n2 = Iris-virginica")
display(results)
print("Accuracy:", round(metrics["accuracy"], 4))
print("Precision:", round(metrics["precision"], 4))
print("Recall:", round(metrics["recall"], 4))
print("F1 (macro):", round(metrics["f1_macro"], 4), "&  F1 (weighted):", round(metrics["f1_weighted"], 4))

Using the best k, which is 3 , the results are as follows:
For labels and predictions:
0 = Iris-setosa
1 = Iris-versicolor
2 = Iris-virginica


,test point,label,prediction
0,"(5.9, 3.0, 5.1, 1.8)",2,2
1,"(6.9, 3.1, 4.9, 1.5)",1,1
2,"(5.3, 3.7, 1.5, 0.2)",0,0
3,"(6.3, 2.8, 5.1, 1.5)",2,1
4,"(6.1, 2.8, 4.7, 1.2)",1,1
5,"(6.8, 3.0, 5.5, 2.1)",2,2
6,"(5.0, 2.0, 3.5, 1.0)",1,1
7,"(6.4, 2.9, 4.3, 1.3)",1,1
8,"(5.2, 4.1, 1.5, 0.1)",0,0
9,"(7.9, 3.8, 6.4, 2.0)",2,2


Accuracy: 0.9333
Precision: 0.9333
Recall: 0.9333
F1 (macro): 0.9373 &  F1 (weighted): 0.9322


Now, we have tried different distances (euclidean and manhattan) and can compare findings on what works best:

In the first iteration, using euclidean distance, the best k for KNN was 1. In the second iteration, using manhattan distance, the best k for KNN was 3. We can see that there is a slight difference here but both distances still achieved a 100% accuracy with the validation data; possibly an overfit. Moving onto test data results, the euclidean distance was able to atain a 97% accuray on the test data while the manhattan distance was only able to achieve a 93% accuracy rating. For precision, euclidean scored slightly higher at 97% compared to manhattan's 93%. Following the same theme, euclidean achieved slightly higher scores for recall, f1(macro), and fl(weighted) at 97%, 97%, and 97% respectively compared to manhattan results at 93%, 94%, and 93%. Given the discussed results,random shuffle of data, and data split chosen in my project, euclidean distance recieves a slight edge.

In [13]:
# Reassign the same training/testing data but make labels = {1,-1} instead of {-1,1} for the perceptron algorithm
simple_training_data =  [([2, 1], 1), ([2, 2], 1), ([3, 2], 1), ([3, 3], 1), ([-1, -1], -1), ([-2, -1], -1), ([-2, -2], -1), ([0, -3], -1)]
simple_testing_data = [([2, 2], 1), ([2, 3], -1), ([-1, -3], -1), ([-3, -3], 1), ([4, 2], 1)]

print("Simple training data:\n",simple_training_data)
print("Simple testing data:\n", simple_testing_data)

Simple training data:
 [([2, 1], 1), ([2, 2], 1), ([3, 2], 1), ([3, 3], 1), ([-1, -1], -1), ([-2, -1], -1), ([-2, -2], -1), ([0, -3], -1)]
Simple testing data:
 [([2, 2], 1), ([2, 3], -1), ([-1, -3], -1), ([-3, -3], 1), ([4, 2], 1)]


In [14]:
#define variables to pass
n_features=2 
learning_rate=1.0
epochs=25

#call simple perceptron and evaluate
b = perceptron(n_features, learning_rate, epochs)
b.fit(simple_training_data)
print("Test accuracy:", b.evaluate(simple_testing_data))
print("Weights:", b.weight)

Test accuracy: 0.6
Weights: [2.0, 1.0, 1.0]


Accuracy is only so low here becuase of the test data I chose where there is 2 points that cleary fit with the majority of points around them they just happen to have different labels. I did this so that it would not just be an easy 100% accuracy

Now we will import the sklearn versions of the knn ML algorithms for KNN and perceptron and compare results

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

SEED = 46

def build_results_df(X, y_true, y_pred, label_map=None):
    df = pd.DataFrame({
        "test_point": [tuple(row) for row in X],
        "label": y_true.astype(int),
        "prediction": y_pred.astype(int)
    })
    if label_map is not None:
        df["label_name"] = df["label"].map(label_map)
        df["prediction_name"] = df["prediction"].map(label_map)
        df = df[["test_point","label","label_name","prediction","prediction_name"]]
    return df

def print_metrics(y_true, y_pred, title="Metrics"):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="micro")   # overall (generic)
    rec  = recall_score(y_true, y_pred,    average="micro")   # overall (generic)
    f1_ma = f1_score(y_true, y_pred,       average="macro")
    f1_wt = f1_score(y_true, y_pred,       average="weighted")

    print(f"{title}\n"
          f"Accuracy: {acc:.4f}\n"
          f"Precision: {prec:.4f}\n"
          f"Recall: {rec:.4f}\n"
          f"F1 (macro): {f1_ma:.4f}   F1 (weighted): {f1_wt:.4f}")


In [ ]:

path = "iris.data"    
cols = ["sepal_length","sepal_width","petal_length","petal_width","species"]
iris = pd.read_csv(path, header=None, names=cols)

# Map species -> int codes 
label_map = {"Iris-setosa":0, "Iris-versicolor":1, "Iris-virginica":2}
inv_label_map = {v:k for k,v in label_map.items()}
iris["label"] = iris["species"].map(label_map).astype(int)

X = iris[["sepal_length","sepal_width","petal_length","petal_width"]].values.astype(float)
y = iris["label"].values.astype(int)

# Split train/val/test (stratified)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, random_state=SEED, stratify=y_trainval
)  
print("Shapes -> train:", X_train.shape, "val:", X_val.shape, "test:", X_test.shape)


Shapes -> train: (90, 4) val: (30, 4) test: (30, 4)


In [ ]:
# Try odd k values; Euclidean by default (metric='minkowski', p=2)
k_grid = list(range(1, 22, 2))

best_k, best_val_acc = None, -1.0
for k in k_grid:
    knn = KNeighborsClassifier(n_neighbors=k, metric="minkowski", p=2)
    knn.fit(X_train, y_train)
    val_acc = knn.score(X_val, y_val)
    if (val_acc > best_val_acc) or (val_acc == best_val_acc and (best_k is None or k < best_k)):
        best_k, best_val_acc = k, val_acc

print(f"Best k on validation: {best_k}  (val acc = {best_val_acc:.4f})")

# Retrain on train+val with best_k, evaluate on test
knn_best = KNeighborsClassifier(n_neighbors=best_k, metric="minkowski", p=2)
knn_best.fit(np.vstack([X_train, X_val]), np.hstack([y_train, y_val]))
y_pred_knn = knn_best.predict(X_test)

# Results & metrics
results_knn = build_results_df(X_test, y_test, y_pred_knn, label_map=inv_label_map)
display(results_knn.head(10))
print_metrics(y_test, y_pred_knn, title="KNN (Euclidean) — Test metrics")

knn_manhattan = KNeighborsClassifier(n_neighbors=best_k, metric="minkowski", p=1)
knn_manhattan.fit(np.vstack([X_train, X_val]), np.hstack([y_train, y_val]))
y_pred_knn_m = knn_manhattan.predict(X_test)
print_metrics(y_test, y_pred_knn_m, title="KNN (Manhattan) — Test metrics")


Best k on validation: 1  (val acc = 1.0000)


,test_point,label,label_name,prediction,prediction_name
0,"(6.4, 3.2, 5.3, 2.3)",2,Iris-virginica,2,Iris-virginica
1,"(6.0, 3.4, 4.5, 1.6)",1,Iris-versicolor,1,Iris-versicolor
2,"(5.5, 2.4, 3.8, 1.1)",1,Iris-versicolor,1,Iris-versicolor
3,"(5.6, 2.7, 4.2, 1.3)",1,Iris-versicolor,1,Iris-versicolor
4,"(5.7, 3.8, 1.7, 0.3)",0,Iris-setosa,0,Iris-setosa
5,"(6.2, 2.9, 4.3, 1.3)",1,Iris-versicolor,1,Iris-versicolor
6,"(6.3, 3.4, 5.6, 2.4)",2,Iris-virginica,2,Iris-virginica
7,"(6.4, 2.8, 5.6, 2.2)",2,Iris-virginica,2,Iris-virginica
8,"(5.7, 2.8, 4.5, 1.3)",1,Iris-versicolor,1,Iris-versicolor
9,"(6.4, 2.8, 5.6, 2.1)",2,Iris-virginica,2,Iris-virginica


KNN (Euclidean) — Test metrics
Accuracy: 0.9667
Precision: 0.9667
Recall: 0.9667
F1 (macro): 0.9666   F1 (weighted): 0.9666


In [ ]:
percep = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", Perceptron(max_iter=1000, tol=1e-3, random_state=SEED))
])
percep.fit(X_train, y_train)           
y_pred_p = percep.predict(X_test)

results_p = build_results_df(X_test, y_test, y_pred_p, label_map=inv_label_map)
display(results_p.head(10))
print_metrics(y_test, y_pred_p, title="Perceptron — Test metrics")


,test_point,label,label_name,prediction,prediction_name
0,"(6.4, 3.2, 5.3, 2.3)",2,Iris-virginica,2,Iris-virginica
1,"(6.0, 3.4, 4.5, 1.6)",1,Iris-versicolor,0,Iris-setosa
2,"(5.5, 2.4, 3.8, 1.1)",1,Iris-versicolor,0,Iris-setosa
3,"(5.6, 2.7, 4.2, 1.3)",1,Iris-versicolor,0,Iris-setosa
4,"(5.7, 3.8, 1.7, 0.3)",0,Iris-setosa,0,Iris-setosa
5,"(6.2, 2.9, 4.3, 1.3)",1,Iris-versicolor,0,Iris-setosa
6,"(6.3, 3.4, 5.6, 2.4)",2,Iris-virginica,2,Iris-virginica
7,"(6.4, 2.8, 5.6, 2.2)",2,Iris-virginica,2,Iris-virginica
8,"(5.7, 2.8, 4.5, 1.3)",1,Iris-versicolor,0,Iris-setosa
9,"(6.4, 2.8, 5.6, 2.1)",2,Iris-virginica,2,Iris-virginica


Perceptron — Test metrics
Accuracy: 0.6000
Precision: 0.6000
Recall: 0.6000
F1 (macro): 0.5106   F1 (weighted): 0.5106


In [19]:
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(y_test, y_pred_knn))
print("\nClassification report (KNN):")
print(classification_report(y_test, y_pred_knn, target_names=[inv_label_map[i] for i in sorted(inv_label_map)]))


Confusion matrix (rows=true, cols=pred):
[[10  0  0]
 [ 0 10  0]
 [ 0  1  9]]

Classification report (KNN):
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       0.91      1.00      0.95        10
 Iris-virginica       1.00      0.90      0.95        10

       accuracy                           0.97        30
      macro avg       0.97      0.97      0.97        30
   weighted avg       0.97      0.97      0.97        30

